In [1]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
import datetime as dt
from sklearn import preprocessing
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Kelsey\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from numpy.random import seed
seed(1)

from tensorflow import random
random.set_seed(2)

In [7]:
# Import the dataset
file_path = Path("../Stephan/Project 2 Sentiment Data - Ethereum2_EthTrader_Vader.csv")
eth_reddit_df = pd.read_csv(file_path, index_col = 'date', parse_dates=True, infer_datetime_format=True)
eth_reddit_df = eth_reddit_df.sort_index()
eth_reddit_df

FileNotFoundError: [Errno 2] File b'..\\Stephan\\Project 2 Sentiment Data - Ethereum2_EthTrader_Vader.csv' does not exist: b'..\\Stephan\\Project 2 Sentiment Data - Ethereum2_EthTrader_Vader.csv'

In [4]:
#eth_bear_reddit_df.insert(4, 'market', 'Bull')
#eth_bear_reddit_df.head()

In [5]:
pip install 'scikit_learn==0.22.2.post1'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'scikit_learn==0.22.2.post1'"


In [6]:
from sklearn.prepocessing import LabelEncoder
#Create the LabelEncoder
le = LabelEncoder()

ModuleNotFoundError: No module named 'sklearn.prepocessing'

In [ ]:
le.fit(eth_reddit_df['Market'])
eth_reddit_df['Market'] = le.transform(eth_reddit_df['Market'])
eth_reddit_df

In [ ]:
#Import the dataset
#file_path = Path("../Project2BKUP/Project 2 Sentiment Data - Ethereum2_EthTrader_Bull.csv")
#eth_bull_reddit_df = pd.read_csv(file_path, index_col = 'date', parse_dates=True, infer_datetime_format=True)
#eth_bull_reddit_df = eth_bull_reddit_df.sort_index()
#eth_bull_reddit_df.head()

In [ ]:
#eth_bull_reddit_df.insert(5, 'market', 'Bull')
#eth_bull_reddit_df.head()

In [ ]:
#eth_reddit_combined = pd.concat([eth_bull_reddit_df, eth_bear_reddit_df], axis="rows", join="inner")
#eth_reddit_combined.head()

In [ ]:
# Drop first row
# by selecting all rows from first row onwards
#start = eth_reddit_combined.index.searchsorted(dt.datetime(2021,3,1))
#end = eth_reddit_combined.index.searchsorted(dt.datetime(2021,5,31))
#eth_reddit_combined

In [ ]:
#ticker_data_slice = ticker_data_grp.loc[2019,4,12]
eth_reddit_df.index.values
#eth_reddit_train = eth_reddit_combined.loc[2021-3-1:2021-5-31]
#eth_reddit_train

In [ ]:
eth_reddit_df.index = pd.to_datetime(eth_reddit_df.index)

In [ ]:
start = eth_reddit_df.index.searchsorted(dt.datetime(2021, 3, 1))
end = eth_reddit_df.index.searchsorted(dt.datetime(2021, 5, 31))
eth_reddit_train = eth_reddit_df.iloc[start:end]
eth_reddit_train

In [ ]:
X = eth_reddit_df["compound"].values
y = eth_reddit_df["Market"].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
y_vader_pred = []
y_vader_prob = []

In [ ]:
#for comment in X_test:
#    y_vader_prob.append(analyzer.polarity_scores(comment)["pos"])
#    sentiment_score = analyzer.polarity_scores(comment)["compound"]
#    if sentiment_score >= 0.1:
#        y_vader_pred.append(1)
#    else:
#        y_vader_pred.append(0)

In [ ]:
# Option 1: Normalizing data using MinMaxScaler from sklearn
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#scaler.fit(np.array(y_vader_prob).reshape(-1,1))
#y_vader_prob_norm = scaler.transform(np.array(y_vader_prob).reshape(-1,1))
#y_vader_prob_norm[:5]

In [ ]:
# Option 2: Using a comprehension list
#normalized = [(x - min(y_vader_prob)) / (max(y_vader_prob) - min(y_vader_prob))
#              for x in y_vader_prob]
#normalized[:5]

In [ ]:
# Import the Tokenizer method from Keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Create an instance of the Tokenizer and fit it with the X text data
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X)

In [ ]:
# Print the first five elements of the encoded vocabulary
for token in list(tokenizer.word_index)[:5]:
    print(f"word: '{token}', token: {tokenizer.word_index[token]}")

In [ ]:
# Transform the text data to numerical sequences
X_seq = tokenizer.texts_to_sequences(X)

In [ ]:
# Contrast a sample numerical sequence with its text version
print("**Text comment**")
print({X[0]})

In [ ]:
print("**Numerical sequence representation**")
print(X_seq[0])

In [ ]:
# Import the pad_sequences method from Keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Set the pad size
max_words = 140

# Pad the sequences using the pad_sequences() method
X_pad = pad_sequences(X_seq, maxlen=max_words, padding="post")

In [ ]:
# Creating training, validation, and testing sets using the encoded data
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_pad, y)

X_train_rnn, X_val_rnn, y_train_rnn, y_val_rnn = train_test_split(X_train_rnn, y_train_rnn)

In [ ]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Model set-up
vocabulary_size = len(tokenizer.word_counts.keys()) + 1
embedding_size = 64

In [ ]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))

# Layer 2
model.add(LSTM(units=280))

# Output layer
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [ ]:
# Show model summary
model.summary()

In [ ]:
# Training the model
batch_size = 1000
epochs = 10
model.fit(
    X_train_rnn,
    y_train_rnn,
    validation_data=(X_val_rnn, y_val_rnn),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

In [ ]:
# Predict classes using the testing data
y_rnn_pred = model.predict_classes(X_test_rnn, batch_size=1000)

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score

#print("Vader Accuracy: %.2f" % (accuracy_score(y_test, y_vader_pred)))
#print("RNN LSTM Accuracy %.2f" % (accuracy_score(y_test_rnn, y_rnn_pred)))

In [ ]:
# Import the confusion_matrix method from sklearn
from sklearn.metrics import confusion_matrix

In [ ]:
# Confusion matrtix metrics from Vader
tn_vader, fp_vader, fn_vader, tp_vader = confusion_matrix(y_test, y_vader_pred).ravel()

# Dataframe to display confusion matrix from Vader
cm_vader_df = pd.DataFrame(
    {
        "Positive(1)": [f"TP={tp_vader}", f"FP={fp_vader}"],
        "Negative(0)": [f"FN={fn_vader}", f"TN={tn_vader}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_vader_df.index.name = "Actual"
cm_vader_df.columns.name = "Predicted"
print("Confusion Matrix from Vader")
display(cm_vader_df)

In [ ]:
# Confusion matrtix metrics from the RNN LSTM model
tn_rnn, fp_rnn, fn_rnn, tp_rnn = confusion_matrix(y_test_rnn, y_rnn_pred).ravel()

# Dataframe to display confusion matrix from the RNN LSTM model
cm_rnn_df = pd.DataFrame(
    {
        "Positive(1)": [f"TP={tp_rnn}", f"FP={fp_rnn}"],
        "Negative(0)": [f"FN={fn_rnn}", f"TN={tn_rnn}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_rnn_df.index.name = "Actual"
cm_rnn_df.columns.name = "Predicted"
print("Confusion Matrix from the RNN LSTM Model")
display(cm_rnn_df)

In [ ]:
# Import the classification_report method from sklearn
from sklearn.metrics import classification_report

In [ ]:
# Display classification report for Vader
print("Classification Report for Vader")
print(classification_report(y_vader_pred, y_test))

In [ ]:
# Display classification report for the RNN LSTM Model
print("Classification Report for the RNN LSTM Model")
print(classification_report(y_rnn_pred, y_test_rnn))